In [1]:
library("caret")
library("caTools")

Loading required package: lattice
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


In [2]:
set.seed(123)
start <- Sys.time()

In [3]:
######### Clinical data
clinical_data <- read.delim("D:/Ders/dicom/NSCLCR01_Clinical_data.txt")
row.names(clinical_data) <- clinical_data[,1]
clinical_data <- clinical_data[,-1]
dim(clinical_data)

[1] 211  39

In [4]:
######### Radiomics data
radiomics_features <- read.delim("D:/Ders/dicom/radiomics_features.txt")
row.names(radiomics_features) <- radiomics_features[,1]
radiomics_features <- radiomics_features[,-1]
dim(radiomics_features)

[1] 87 32

In [5]:
######### Genetic data
genetic_data_first <- read.delim("D:/Ders/dicom/GSE103584_R01_NSCLC_RNAseq.txt")
genetic_data <- t(genetic_data_first)
genetic_data <- as.data.frame(genetic_data)
colnames(genetic_data) <- genetic_data_first[,1]
genetic_data <- genetic_data[-1,]
dim(genetic_data)

[1]   130 22126

In [11]:
######### Removing columns containing NA in genetic data
na_genes <- NULL
for(i in 1:dim(genetic_data)[2])
{
  if(sum(is.na(genetic_data[,i]))>0){
    na_genes <- c(na_genes, names(genetic_data)[i])
  }
}

genetic_data_filtered <- subset(genetic_data, select = ! names(genetic_data) %in% na_genes)
dim(genetic_data_filtered)

row.names(genetic_data_filtered) <- gsub('R01.', 'R01-', row.names(genetic_data_filtered))
genetic_data_filtered <- as.data.frame(genetic_data_filtered)
genetic_data_filtered <- as.matrix(genetic_data_filtered)
storage.mode(genetic_data_filtered) <- "double"

[1]  130 5268

In [12]:
######### Selecting common samples for all data sets
common_samples <- intersect(row.names(radiomics_features), row.names(genetic_data_filtered))
radiomics_features_common <- radiomics_features[rownames(radiomics_features) %in% common_samples, ]  # Extract rows from data
genetic_data_common <- genetic_data_filtered[rownames(genetic_data_filtered) %in% common_samples, ]
clinical_data_common <- clinical_data[rownames(clinical_data) %in% common_samples, ]

nsclc_nos <- row.names(clinical_data_common[clinical_data_common$Histology=="NSCLC NOS (not otherwise specified)",])
radiomics_features_last <- radiomics_features_common[!rownames(radiomics_features_common) %in% nsclc_nos, ] 
genetic_data_last <- genetic_data_common[!rownames(genetic_data_common) %in% nsclc_nos, ] 
clinical_data_last <- clinical_data_common[!rownames(clinical_data_common) %in% nsclc_nos, ] 

data <- cbind(clinical_data_last$Histology, radiomics_features_last, genetic_data_last)
colnames(data)[1] <- "Histology"
dim(data)

[1]   58 5301

In [15]:
######### Data split
index=sample.split(data$Histology,SplitRatio=0.7)
train=data[index,]
test=data[!index,]

In [17]:
######### Cross-validation
ctrl = trainControl(method="cv", number=5, returnResamp = "final", summaryFunction = prSummary, classProbs = TRUE,
                    savePredictions = T, verbose=F)

train$Histology <- as.factor(train$Histology)
train$Histology <- droplevels(train$Histology)
summary(train$Histology)
class(train$Histology)
levels(train$Histology) <- gsub(" ", "_", levels(train$Histology))
levels(train$Histology)


test$Histology <- as.factor(test$Histology)
test$Histology <- droplevels(test$Histology)
summary(test$Histology)
class(test$Histology)
levels(test$Histology) <- gsub(" ", "_", levels(test$Histology))
levels(test$Histology)


Adenocarcinoma Squamous_cell_carcinoma 
                     29                      11

[1] "factor"

[1] "Adenocarcinoma"          "Squamous_cell_carcinoma"

Adenocarcinoma Squamous_cell_carcinoma 
                     13                       5

[1] "factor"

[1] "Adenocarcinoma"          "Squamous_cell_carcinoma"

In [1]:
confMat<- function(data, ...) UseMethod("confMat")

confMat.default<-function(data, reference, positive = NULL, verbose = TRUE, ...) {

  if(!is.factor(data)) data <- factor(data)
  if(!is.factor(reference)) reference <- factor(reference)
  
  
  if(length(levels(reference))!=2)
    stop("The reference must have 2 factor levels.")
  
  if(any(levels(reference) != levels(data))) {
    warning("Levels are not in the same order for reference and data. Refactoring data to match.")
    data <- as.character(data)
    data <- factor(data, levels = levels(reference))
  }
  
  store <- table(data,reference)
  
  if (!is.null(positive)){
  if (which(colnames(store)==positive)==2) {

temp <- store[1,]
store[1,] <- store [2,]
store[2,] <- temp
temp <- store[,1]
store[,1] <- store[,2]
store[,2] <- temp
temp <- colnames(store)[1]
colnames(store)[1] <- colnames(store)[2]
colnames(store)[2] <- temp
rownames(store) <- colnames(store)

  }}else{positive <- colnames(store)[1]}
    
  
  accuracy <- sum(diag(store))/sum(store)
  
  pc <- sum(diag(rowSums(store) %*% t(colSums(store))))/(sum(store)^2)
  
  kappa <- (accuracy-pc)/(1-pc) 
  
  
  denom <-colSums(store)[1]*colSums(store)[2]*rowSums(store)[1]*rowSums(store)[2]

  MCC <- (store[1,1]*store[2,2]-store[1,2]*store[2,1])/sqrt(denom)

  ccr <- diag(store)/colSums(store)
  
  sensitivity <- ccr[1]
  specificity <- ccr[2]
  
  pv <- diag(store)/rowSums(store)
  prevalences <- colSums(store)/sum(store)
  NIR <- max(prevalences)
  prevalence <- prevalences[1]
  PPV <- pv[1]
  NPV <- pv[2]
  baccuracy <- (sensitivity+specificity)/2
  youden <- sensitivity+specificity-1
  detectRate <- store[1,1]/sum(store)
  detectPrevalence <- rowSums(store)[1]/sum(store)
  precision <- PPV
  recall <- sensitivity
  F1 <- 2/(1/recall+1/precision)
    
  if (verbose) {
    cat("\n")
    cat("Confusion Matrix and Statistics", "\n\n", sep = " ")
    print(store)
    cat("\n")
    cat("\n", "    Accuracy             :  ", round(accuracy,4), sep = " ")
    cat("\n", "    No Information Rate  :  ", round(NIR,4), sep = " ")
    cat("\n", "    Kappa                :  ", round(kappa,4), sep = " ")
    cat("\n", "    Matthews Corr Coef   :  ", round(MCC,4), sep = " ")
    cat("\n", "    Sensitivity          :  ", round(sensitivity,4), sep = " ")
    cat("\n", "    Specificity          :  ", round(specificity,4), sep = " ")
    cat("\n", "    Positive Pred Value  :  ", round(PPV,4), sep = " ")
    cat("\n", "    Negative Pred Value  :  ", round(NPV,4), sep = " ")
    cat("\n", "    Prevalence           :  ", round(prevalence,4), sep = " ")
    cat("\n", "    Balanced Accuracy    :  ", round(baccuracy,4), sep = " ")
    cat("\n", "    Youden Index         :  ", round(youden,4), sep = " ")
    cat("\n", "    Detection Rate       :  ", round(detectRate,4), sep = " ")
    cat("\n", "    Detection Prevalence :  ", round(detectPrevalence,4), sep = " ")
    cat("\n", "    Precision            :  ", round(precision,4), sep = " ")
    cat("\n", "    Recall               :  ", round(recall,4), sep = " ")
    cat("\n", "    F1                   :  ", round(F1,4), "\n", sep = " ")
    cat("\n", "    Positive Class       :  ", positive, "\n", sep = " ")
    cat("\n")
  }   
  
  all<-rbind(as.numeric(accuracy),as.numeric(NIR),as.numeric(kappa),as.numeric(MCC),as.numeric(sensitivity),as.numeric(specificity),as.numeric(PPV),as.numeric(NPV),as.numeric(prevalence),as.numeric(baccuracy),as.numeric(youden),as.numeric(detectRate),as.numeric(detectPrevalence),as.numeric(precision),as.numeric(recall),as.numeric(F1))

  rownames(all)<-c("accuracy","NIR","kappa","MCC","sensitivity","specificity","PPV","NPV","prevalence","baccuracy","youden","detectRate","detectPrev","precision","recall","F1")
colnames(all)<-""
 
  result <- list()
  result$table <- store
  result$accuracy <- as.numeric(accuracy)
  result$NIR <- as.numeric(NIR)
  result$kappa <- as.numeric(kappa)
  result$MCC <- as.numeric(MCC)
  result$sensitivity <- as.numeric(sensitivity)
  result$specificity <- as.numeric(specificity)
  result$PPV <- as.numeric(PPV)
  result$NPV <- as.numeric(NPV)
  result$prevalence <- as.numeric(prevalence)
  result$baccuracy <- as.numeric(baccuracy)
  result$youden <- as.numeric(youden)
  result$detectRate <- as.numeric(detectRate)
  result$detectPrevalence <- as.numeric(detectPrevalence)
  result$precision <- as.numeric(precision)
  result$recall <- as.numeric(recall)
  result$F1 <- as.numeric(F1)
  result$all <- all
  
  invisible(result)
  
}


confMat.table<-function(data, positive = NULL, verbose = TRUE, ...) {

  if(length(dim(data)) != 2) stop("The table must have two dimensions.")
  if(nrow(data) != ncol(data)) stop("The number of rows must be equal to the number of columns.")
  if(length(colnames(data))!=2) stop("The reference must have 2 factor levels.")

  store <- data
  
  if (!is.null(positive)){
  if (which(colnames(store)==positive)==2) {

temp <- store[1,]
store[1,] <- store [2,]
store[2,] <- temp
temp <- store[,1]
store[,1] <- store[,2]
store[,2] <- temp
temp <- colnames(store)[1]
colnames(store)[1] <- colnames(store)[2]
colnames(store)[2] <- temp
rownames(store) <- colnames(store)

  }}else{positive <- colnames(store)[1]}   
  
  accuracy <- sum(diag(store))/sum(store)
  
  pc <- sum(diag(rowSums(store) %*% t(colSums(store))))/(sum(store)^2)
  
  kappa <- (accuracy-pc)/(1-pc) 
  
  denom <-colSums(store)[1]*colSums(store)[2]*rowSums(store)[1]*rowSums(store)[2]

  MCC <- (store[1,1]*store[2,2]-store[1,2]*store[2,1])/sqrt(denom)

  ccr <- diag(store)/colSums(store)
  
  sensitivity <- ccr[1]
  specificity <- ccr[2]
  
  pv <- diag(store)/rowSums(store)
  prevalences <- colSums(store)/sum(store)
  NIR <- max(prevalences)
  prevalence <- prevalences[1]
  PPV <- pv[1]
  NPV <- pv[2]
  baccuracy <- (sensitivity+specificity)/2
  youden <- sensitivity+specificity-1
  detectRate <- store[1,1]/sum(store)
  detectPrevalence <- rowSums(store)[1]/sum(store)
  precision <- PPV
  recall <- sensitivity
  F1 <- 2/(1/recall+1/precision)
    
  if (verbose) {
    cat("\n")
    cat("Confusion Matrix and Statistics", "\n\n", sep = " ")
    print(store)
    cat("\n")
    cat("\n", "    Accuracy             :  ", round(accuracy,4), sep = " ")
    cat("\n", "    No Information Rate  :  ", round(NIR,4), sep = " ")
    cat("\n", "    Kappa                :  ", round(kappa,4), sep = " ")
    cat("\n", "    Matthews Corr Coef   :  ", round(MCC,4), sep = " ")
    cat("\n", "    Sensitivity          :  ", round(sensitivity,4), sep = " ")
    cat("\n", "    Specificity          :  ", round(specificity,4), sep = " ")
    cat("\n", "    Positive Pred Value  :  ", round(PPV,4), sep = " ")
    cat("\n", "    Negative Pred Value  :  ", round(NPV,4), sep = " ")
    cat("\n", "    Prevalence           :  ", round(prevalence,4), sep = " ")
    cat("\n", "    Balanced Accuracy    :  ", round(baccuracy,4), sep = " ")
    cat("\n", "    Youden Index         :  ", round(youden,4), sep = " ")
    cat("\n", "    Detection Rate       :  ", round(detectRate,4), sep = " ")
    cat("\n", "    Detection Prevalence :  ", round(detectPrevalence,4), sep = " ")
    cat("\n", "    Precision            :  ", round(precision,4), sep = " ")
    cat("\n", "    Recall               :  ", round(recall,4), sep = " ")
    cat("\n", "    F1                   :  ", round(F1,4), "\n", sep = " ")
    cat("\n", "    Positive Class       :  ", positive, "\n", sep = " ")
    cat("\n")
  }   
  
  all<-rbind(as.numeric(accuracy),as.numeric(NIR),as.numeric(kappa),as.numeric(MCC),as.numeric(sensitivity),as.numeric(specificity),as.numeric(PPV),as.numeric(NPV),as.numeric(prevalence),as.numeric(baccuracy),as.numeric(youden),as.numeric(detectRate),as.numeric(detectPrevalence),as.numeric(precision),as.numeric(recall),as.numeric(F1))

rownames(all)<-c("accuracy","NIR","kappa","MCC","sensitivity","specificity","PPV","NPV","prevalence","baccuracy","youden","detectRate","detectPrev","precision","recall","F1")
colnames(all)<-""
  
  result <- list()
  result$table <- store
  result$accuracy <- as.numeric(accuracy)
  result$NIR <- as.numeric(NIR)
  result$kappa <- as.numeric(kappa)
  result$MCC <- as.numeric(MCC)
  result$sensitivity <- as.numeric(sensitivity)
  result$specificity <- as.numeric(specificity)
  result$PPV <- as.numeric(PPV)
  result$NPV <- as.numeric(NPV)
  result$prevalence <- as.numeric(prevalence)
  result$baccuracy <- as.numeric(baccuracy)
  result$youden <- as.numeric(youden)
  result$detectRate <- as.numeric(detectRate)
  result$detectPrev <- as.numeric(detectPrevalence)
  result$precision <- as.numeric(precision)
  result$recall <- as.numeric(recall)
  result$F1 <- as.numeric(F1)
  result$all <- all

  invisible(result)


}

In [18]:
############################################# Without feature selection#############################################
######### Elastic net
model_elastic_radiomics = train(Histology~., data = train[,1:33], method = "glmnet", trControl = ctrl, tuneLength = 5)
pred_en_radiomics <- predict(model_elastic_radiomics, test[,-1])
perf_en_radiomics <- confMat(pred_en_radiomics, test$Histology, verbose = FALSE)$all

model_elastic_genomics = train(Histology~., data = train[,c(1,34:5301)], method = "glmnet", trControl = ctrl, tuneLength = 5)
pred_en_genomics  <- predict(model_elastic_genomics , test[,-1])
perf_en_genomics  <- confMat(pred_en_genomics , test$Histology, verbose = FALSE)$all

model_elastic_all = train(Histology~., data = train, method = "glmnet", trControl = ctrl, tuneLength = 5)
pred_en_all <- predict(model_elastic_all, test[,-1])
perf_en_all <- confMat(pred_en_all, test$Histology, verbose = FALSE)$all

######### Random forest
model_rf_radiomics = train(Histology~., data = train[,1:33], method = "rf", trControl = ctrl, tuneLength = 5)
pred_rf_radiomics <- predict(model_rf_radiomics, test[,-1])
perf_rf_radiomics <- confMat(pred_rf_radiomics, test$Histology, verbose = FALSE)$all

model_rf_genomics = train(Histology~., data = train[,c(1,34:5301)], method = "rf", trControl = ctrl, tuneLength = 5)
pred_rf_genomics  <- predict(model_rf_genomics , test[,-1])
perf_rf_genomics  <- confMat(pred_rf_genomics , test$Histology, verbose = FALSE)$all

model_rf_all = train(Histology~., data = train, method = "rf", trControl = ctrl, tuneLength = 5)
pred_rf_all <- predict(model_rf_all, test[,-1])
perf_rf_all <- confMat(pred_rf_all, test$Histology, verbose = FALSE)$all

######### Support vector machines
model_svm_linear_radiomics <- train(Histology~., data = train[,1:33], method = "svmLinear", trControl = ctrl, tuneLength = 5)
pred_svm_linear_radiomics <- predict(model_svm_linear_radiomics, test[,-1])
perf_svm_linear_radiomics <- confMat(pred_svm_linear_radiomics, test$Histology, verbose = FALSE)$all

model_svm_linear_genomics <- train(Histology~., data = train[,c(1,34:5301)], method = "svmLinear", trControl = ctrl, tuneLength = 5)
pred_svm_linear_genomics <- predict(model_svm_linear_genomics, test[,-1])
perf_svm_linear_genomics <- confMat(pred_svm_linear_genomics, test$Histology, verbose = FALSE)$all

model_svm_linear_all <- train(Histology~., data = train, method = "svmLinear", trControl = ctrl, tuneLength = 5)
pred_svm_linear_all <- predict(model_svm_linear_all, test[,-1])
perf_svm_linear_all <- confMat(pred_svm_linear_all, test$Histology, verbose = FALSE)$all


model_svm_radial_radiomics <- train(Histology~., data = train[,1:33], method = "svmRadial", trControl = ctrl, tuneLength = 5)
pred_svm_radial_radiomics <- predict(model_svm_radial_radiomics, test[,-1])
perf_svm_radial_radiomics <- confMat(pred_svm_radial_radiomics, test$Histology, verbose = FALSE)$all

model_svm_radial_genomics <- train(Histology~., data = train[,c(1,34:5301)], method = "svmRadial", trControl = ctrl, tuneLength = 5)
pred_svm_radial_genomics <- predict(model_svm_radial_genomics, test[,-1])
perf_svm_radial_genomics <- confMat(pred_svm_radial_genomics, test$Histology, verbose = FALSE)$all

model_svm_radial_all <- train(Histology~., data = train, method = "svmRadial", trControl = ctrl, tuneLength = 5)
pred_svm_radial_all <- predict(model_svm_radial_all, test[,-1])
perf_svm_radial_all <- confMat(pred_svm_radial_all, test$Histology, verbose = FALSE)$all


model_svm_poly_radiomics <- train(Histology~., data = train[,1:33], method = "svmPoly", trControl = ctrl, tuneLength = 5) 
pred_svm_poly_radiomics <- predict(model_svm_poly_radiomics, test[,-1])
perf_svm_poly_radiomics <- confMat(pred_svm_poly_radiomics, test$Histology, verbose = FALSE)$all

model_svm_poly_genomics <- train(Histology~., data = train[,c(1,34:5301)], method = "svmPoly", trControl = ctrl, tuneLength = 5)
pred_svm_poly_genomics <- predict(model_svm_poly_genomics, test[,-1])
perf_svm_poly_genomics <- confMat(pred_svm_poly_genomics, test$Histology, verbose = FALSE)$all

model_svm_poly_all <- train(Histology~., data = train, method = "svmPoly", trControl = ctrl, tuneLength = 5)
pred_svm_poly_all <- predict(model_svm_poly_all, test[,-1])
perf_svm_poly_all <- confMat(pred_svm_poly_all, test$Histology, verbose = FALSE)$all

######### Artificial Neural Network
model_ann_radiomics = train(Histology~., data = train[,1:33], method = "nnet", trControl = ctrl, trace = FALSE, tuneLength = 5)
pred_ann_radiomics <- predict(model_ann_radiomics, test[,-1])
perf_ann_radiomics <- confMat(pred_ann_radiomics, test$Histology, verbose = FALSE)$all

model_ann_genomics = train(Histology~., data = train[,c(1,34:5301)], method = "nnet", trControl = ctrl, trace = FALSE, tuneLength = 5)
pred_ann_genomics  <- predict(model_ann_genomics , test[,-1])
perf_ann_genomics  <- confMat(pred_ann_genomics , test$Histology, verbose = FALSE)$all

model_ann_all = train(Histology~., data = train,  trControl = ctrl, trace = FALSE, tuneLength = 5)
pred_ann_all <- predict(model_ann_all, test[,-1])
perf_ann_all <- confMat(pred_ann_all, test$Histology, verbose = FALSE)$all

######### Xgboost
model_xgboost_radiomics = train(Histology~., data = train[,1:33], method = "xgbTree", trControl = ctrl, tuneLength = 5)
pred_xgboost_radiomics <- predict(model_xgboost_radiomics, test[,-1])
perf_xgboost_radiomics <- confMat(pred_xgboost_radiomics, test$Histology, verbose = FALSE)$all

model_xgboost_genomics = train(Histology~., data = train[,c(1,34:5301)], method = "xgbTree", trControl = ctrl, tuneLength = 5)
pred_xgboost_genomics  <- predict(model_xgboost_genomics , test[,-1])
perf_xgboost_genomics  <- confMat(pred_xgboost_genomics , test$Histology, verbose = FALSE)$all

model_xgboost_all = train(Histology~., data = train, method = "xgbTree", trControl = ctrl, tuneLength = 5)
pred_xgboost_all <- predict(model_xgboost_all, test[,-1])
perf_xgboost_all <- confMat(pred_xgboost_all, test$Histology, verbose = FALSE)$all

results <- cbind(perf_en_radiomics, perf_en_genomics, perf_en_all, 
                 perf_rf_radiomics, perf_rf_genomics, perf_rf_all, 
                 perf_svm_linear_radiomics, perf_svm_linear_genomics, perf_svm_linear_all, 
                 perf_svm_radial_radiomics, perf_svm_radial_genomics, perf_svm_radial_all, 
                 perf_svm_poly_radiomics, perf_svm_poly_genomics, perf_svm_poly_all, 
                 perf_ann_radiomics,#perf_ann_genomics, perf_ann_all, 
                 perf_xgboost_radiomics, perf_xgboost_genomics, perf_xgboost_all)


colnames(results) <- c("Elastic  net radiomics", "Elastic  net genomics", "Elastic  net all", 
                       "Random forest radiomics", "Random forest genomics", "Random forest all", 
                       "SVM-linear radiomics", "SVM-linear genomics", "SVM-linear all", 
                       "SVM-radial radiomics", "SVM-radial genomics", "SVM-radial all",  
                       "SVM-poly radiomics", "SVM-poly genomics", "SVM-poly all", 
                       "ANN radiomics",#"ANN genomics", "ANN all", 
                       "Xgboost radiomics", "Xgboost genomics", "Xgboost all")

write.csv(results, "Results_without_feature_selection_set.seed_123.csv")



Warning message in train.default(x, y, weights = w, ...):
"The metric "Accuracy" was not in the result set. AUC will be used instead."Warning message:
"from glmnet Fortran code (error code -93); Convergence for 93th lambda value not reached after maxit=100000 iterations; solutions for larger lambdas returned"Warning message:
"from glmnet Fortran code (error code -86); Convergence for 86th lambda value not reached after maxit=100000 iterations; solutions for larger lambdas returned"Warning message in train.default(x, y, weights = w, ...):
"The metric "Accuracy" was not in the result set. AUC will be used instead."Warning message in train.default(x, y, weights = w, ...):
"The metric "Accuracy" was not in the result set. AUC will be used instead."

In [40]:
############################################################# RFE #############################################################
control_rfe = rfeControl(functions = rfFuncs, #treebagFuncs , #nbFuncs, #, # random forest
                         method = "repeatedcv", # repeated cv
                         number = 5, # number of folds
                         repeats = 5) # number of repeats
                         
# Performing RFE
radiomics <- train[,1:33]
result_rfe_radiomics = rfe(x = radiomics[,-1], #x_train, 
                           y = as.factor(radiomics[,1]), #y_train,
                           sizes = c(1:8), rfeControl = control_rfe)
result_rfe_radiomics$optVariables[1:5] 
radiomics_rfe <- cbind(as.factor(radiomics[,1]), subset(radiomics, select = result_rfe_radiomics$optVariables[1:5]))
names(radiomics_rfe)[1] <- "Histology"


genomics <- train[,c(1,34:5301)]
result_rfe_genomics = rfe(x = genomics[,-1], #x_train, 
                          y = genomics[,1], #y_train,
                          sizes = c(1:8), rfeControl = control_rfe)
result_rfe_genomics$optVariables[1:5] 
genomics_rfe <- cbind(as.factor(genomics[,1]), subset(genomics, select = result_rfe_genomics$optVariables[1:5]))
names(genomics_rfe)[1] <- "Histology"

all_rfe <- cbind(radiomics_rfe, genomics_rfe[,-1])

########## Elastic Net
model_en_rfe_radiomics = train(Histology~., data = radiomics_rfe, method = "glmnet", trControl = ctrl, tuneLength = 5)
pred_en_rfe_radiomics <- predict(model_en_rfe_radiomics, test[,-1])
perf_en_rfe_radiomics <- confMat(pred_en_rfe_radiomics, test$Histology, verbose = FALSE)$all

model_en_rfe_genomics = train(Histology~., data = genomics_rfe, method = "glmnet", trControl = ctrl, tuneLength = 5)
pred_en_rfe_genomics <- predict(model_en_rfe_genomics, test[,-1])
perf_en_rfe_genomics <- confMat(pred_en_rfe_genomics, test$Histology, verbose = FALSE)$all

model_en_rfe_all = train(Histology~., data = all_rfe, method = "glmnet", trControl = ctrl, tuneLength = 5)
pred_en_rfe_all <- predict(model_en_rfe_all, test[,-1])
perf_en_rfe_all <- confMat(pred_en_rfe_all, test$Histology, verbose = FALSE)$all

########## Random Forest
model_rf_rfe_radiomics = train(Histology~., data = radiomics_rfe, method = "rf", trControl = ctrl, tuneLength = 5)
pred_rf_rfe_radiomics <- predict(model_rf_rfe_radiomics, test[,-1])
perf_rf_rfe_radiomics <- confMat(pred_rf_rfe_radiomics, test$Histology, verbose = FALSE)$all

model_rf_rfe_genomics = train(Histology~., data = genomics_rfe, method = "rf", trControl = ctrl, tuneLength = 5)
pred_rf_rfe_genomics <- predict(model_rf_rfe_genomics, test[,-1])
perf_rf_rfe_genomics <- confMat(pred_rf_rfe_genomics, test$Histology, verbose = FALSE)$all

model_rf_rfe_all = train(Histology~., data = all_rfe, method = "rf", trControl = ctrl, tuneLength = 5)
pred_rf_rfe_all <- predict(model_rf_rfe_all, test[,-1])
perf_rf_rfe_all <- confMat(pred_rf_rfe_all, test$Histology, verbose = FALSE)$all

########## Support Vector Machines - Linear
model_svm_linear_rfe_radiomics = train(Histology~., data = radiomics_rfe, method = "svmLinear", trControl = ctrl, tuneLength = 5)
pred_svm_linear_rfe_radiomics <- predict(model_svm_linear_rfe_radiomics, test[,-1])
perf_svm_linear_rfe_radiomics <- confMat(pred_svm_linear_rfe_radiomics, test$Histology, verbose = FALSE)$all

model_svm_linear_rfe_genomics = train(Histology~., data = genomics_rfe, method = "svmLinear", trControl = ctrl, tuneLength = 5)
pred_svm_linear_rfe_genomics <- predict(model_svm_linear_rfe_genomics, test[,-1])
perf_svm_linear_rfe_genomics <- confMat(pred_svm_linear_rfe_genomics, test$Histology, verbose = FALSE)$all

model_svm_linear_rfe_all = train(Histology~., data = all_rfe, method = "svmLinear", trControl = ctrl, tuneLength = 5)
pred_svm_linear_rfe_all <- predict(model_svm_linear_rfe_all, test[,-1])
perf_svm_linear_rfe_all <- confMat(pred_svm_linear_rfe_all, test$Histology, verbose = FALSE)$all

########## Support Vector Machines - Radial
model_svm_radial_rfe_radiomics = train(Histology~., data = radiomics_rfe, method = "svmRadial", trControl = ctrl, tuneLength = 5)
pred_svm_radial_rfe_radiomics <- predict(model_svm_radial_rfe_radiomics, test[,-1])
perf_svm_radial_rfe_radiomics <- confMat(pred_svm_radial_rfe_radiomics, test$Histology, verbose = FALSE)$all

model_svm_radial_rfe_genomics = train(Histology~., data = genomics_rfe, method = "svmRadial", trControl = ctrl, tuneLength = 5)
pred_svm_radial_rfe_genomics <- predict(model_svm_radial_rfe_genomics, test[,-1])
perf_svm_radial_rfe_genomics <- confMat(pred_svm_radial_rfe_genomics, test$Histology, verbose = FALSE)$all

model_svm_radial_rfe_all = train(Histology~., data = all_rfe, method = "svmRadial", trControl = ctrl, tuneLength = 5)
pred_svm_radial_rfe_all <- predict(model_svm_radial_rfe_all, test[,-1])
perf_svm_radial_rfe_all <- confMat(pred_svm_radial_rfe_all, test$Histology, verbose = FALSE)$all

########## Support Vector Machines - Polynomial
model_svm_poly_rfe_radiomics = train(Histology~., data = radiomics_rfe, method = "svmPoly", trControl = ctrl, tuneLength = 5)
pred_svm_poly_rfe_radiomics <- predict(model_svm_poly_rfe_radiomics, test[,-1])
perf_svm_poly_rfe_radiomics <- confMat(pred_svm_poly_rfe_radiomics, test$Histology, verbose = FALSE)$all

model_svm_poly_rfe_genomics = train(Histology~., data = genomics_rfe, method = "svmPoly", trControl = ctrl, tuneLength = 5)
pred_svm_poly_rfe_genomics <- predict(model_svm_poly_rfe_genomics, test[,-1])
perf_svm_poly_rfe_genomics <- confMat(pred_svm_poly_rfe_genomics, test$Histology, verbose = FALSE)$all

model_svm_poly_rfe_all = train(Histology~., data = all_rfe, method = "svmPoly", trControl = ctrl, tuneLength = 5)
pred_svm_poly_rfe_all <- predict(model_svm_poly_rfe_all, test[,-1])
perf_svm_poly_rfe_all <- confMat(pred_svm_poly_rfe_all, test$Histology, verbose = FALSE)$all

########## Artificial Neural Networks
model_ann_rfe_radiomics = train(Histology~., data = radiomics_rfe, method = "nnet", trControl = ctrl, trace = FALSE, tuneLength = 5)
pred_ann_rfe_radiomics <- predict(model_ann_rfe_radiomics, test[,-1])
perf_ann_rfe_radiomics <- confMat(pred_ann_rfe_radiomics, test$Histology, verbose = FALSE)$all

model_ann_rfe_genomics = train(Histology~., data = genomics_rfe, method = "nnet", trControl = ctrl, trace = FALSE, tuneLength = 5)
pred_ann_rfe_genomics <- predict(model_ann_rfe_genomics, test[,-1])
perf_ann_rfe_genomics <- confMat(pred_ann_rfe_genomics, test$Histology, verbose = FALSE)$all

model_ann_rfe_all = train(Histology~., data = all_rfe, method = "nnet", trControl = ctrl, trace = FALSE, tuneLength = 5)
pred_ann_rfe_all <- predict(model_ann_rfe_all, test[,-1])
perf_ann_rfe_all <- confMat(pred_ann_rfe_all, test$Histology, verbose = FALSE)$all

########## XGBoost
model_xgboost_rfe_radiomics = train(Histology~., data = radiomics_rfe, method = "xgbTree", trControl = ctrl, tuneLength = 5)
pred_xgboost_rfe_radiomics <- predict(model_xgboost_rfe_radiomics, test[,-1])
perf_xgboost_rfe_radiomics <- confMat(pred_xgboost_rfe_radiomics, test$Histology, verbose = FALSE)$all

model_xgboost_rfe_genomics = train(Histology~., data = genomics_rfe, method = "xgbTree", trControl = ctrl, tuneLength = 5)
pred_xgboost_rfe_genomics <- predict(model_xgboost_rfe_genomics, test[,-1])
perf_xgboost_rfe_genomics <- confMat(pred_xgboost_rfe_genomics, test$Histology, verbose = FALSE)$all

model_xgboost_rfe_all = train(Histology~., data = all_rfe, method = "xgbTree", trControl = ctrl, tuneLength = 5)
pred_xgboost_rfe_all <- predict(model_xgboost_rfe_all, test[,-1])
perf_xgboost_rfe_all <- confMat(pred_xgboost_rfe_all, test$Histology, verbose = FALSE)$all


results <- cbind(perf_en_rfe_radiomics, perf_en_rfe_genomics, perf_en_rfe_all, 
                 perf_rf_rfe_radiomics, perf_rf_rfe_genomics, perf_rf_rfe_all, 
                 perf_svm_linear_rfe_radiomics, perf_svm_linear_rfe_genomics, perf_svm_linear_rfe_all, 
                 perf_svm_radial_rfe_radiomics, perf_svm_radial_rfe_genomics, perf_svm_radial_rfe_all, 
                 perf_svm_poly_rfe_radiomics, perf_svm_poly_rfe_genomics, perf_svm_poly_rfe_all, 
                 perf_ann_rfe_radiomics, perf_ann_rfe_genomics, perf_ann_rfe_all, 
                 perf_xgboost_rfe_radiomics, perf_xgboost_rfe_genomics, perf_xgboost_rfe_all)


colnames(results) <- c("Elastic  net radiomics", "Elastic  net genomics", "Elastic net all", 
                       "Random forest radiomics", "Random forest genomics", "Random forest all", 
                       "SVM-linear radiomics", "SVM-linear genomics", "SVM-linear all", 
                       "SVM-radial radiomics", "SVM-radial genomics", "SVM-radial all",  
                       "SVM-poly radiomics", "SVM-poly genomics",  "SVM-poly all", 
                       "ANN radiomics", "ANN genomics", "ANN all", 
                       "Xgboost radiomics", "Xgboost genomics", "Xgboost all")

write.csv(results, "Result_RFE_set.seed_123.csv")
View(results)

[1] "X30" "X7"  "X8"  "X15" "X13"

[1] "GTF3A"   "DNAJB14" "ZCCHC2"  "MYLIP"   "SETD5"

In [53]:
################################################### Univariate analysis ###################################################
###### AUC
AUC_results <- colAUC(train[,2:33], train$Histology)
AUC_names <- store <- NULL
for(i in 1:length(AUC_results)){
  if(AUC_results[1,][i]>0.65){
    AUC_names <- names(AUC_results[1,][i])
    store <- cbind(store, AUC_names)
  }
}
length(store)
radiomics_data_AUC <- subset(train[,2:33], select = store[1,])
dim(radiomics_data_AUC)
radiomics_data_AUC_Histology <- cbind(train$Histology, radiomics_data_AUC)
names(radiomics_data_AUC_Histology)[1] <- "Histology"

#########
AUC_results <- colAUC(train[,34:5301], train$Histology)
AUC_names <- store <- NULL
for(i in 1:length(AUC_results)){
  if(AUC_results[1,][i]>0.70){
    AUC_names <- names(AUC_results[1,][i])
    store <- cbind(store, AUC_names)
  }
}
length(store)

genetic_data_AUC <- subset(train[,34:5301], select = store[1,])
dim(genetic_data_AUC)
genetic_data_AUC_histology <- cbind(train$Histology, genetic_data_AUC)
names(genetic_data_AUC_histology)[1] <- "Histology"

all_rfe <- cbind(radiomics_data_AUC_Histology, genetic_data_AUC_histology[,-1])

#Cross-validation
ctrl = trainControl(method="cv", number=5, returnResamp = "final", summaryFunction = prSummary, classProbs = TRUE,
                    savePredictions = T, verbose=F)

########## Elastic Net
model_en_rfe_radiomics = train(Histology~., data = radiomics_data_AUC_Histology, method = "glmnet", trControl = ctrl, tuneLength=5)
pred_en_rfe_radiomics <- predict(model_en_rfe_radiomics, test[,-1])
perf_en_rfe_radiomics <- confMat(pred_en_rfe_radiomics, test$Histology, verbose = FALSE)$all

model_en_rfe_genomics = train(Histology~., data = genetic_data_AUC_histology, method = "glmnet", trControl = ctrl, tuneLength=5)
pred_en_rfe_genomics <- predict(model_en_rfe_genomics, test[,-1])
perf_en_rfe_genomics <- confMat(pred_en_rfe_genomics, test$Histology, verbose = FALSE)$all

model_en_rfe_all = train(Histology~., data = all_rfe, method = "glmnet", trControl = ctrl, tuneLength=5)
pred_en_rfe_all <- predict(model_en_rfe_all, test[,-1])
perf_en_rfe_all <- confMat(pred_en_rfe_all, test$Histology, verbose = FALSE)$all


########## Random Forest
model_rf_rfe_radiomics = train(Histology~., data = radiomics_data_AUC_Histology, method = "rf", trControl = ctrl, tuneLength=5)
pred_rf_rfe_radiomics <- predict(model_rf_rfe_radiomics, test[,-1])
perf_rf_rfe_radiomics <- confMat(pred_rf_rfe_radiomics, test$Histology, verbose = FALSE)$all

model_rf_rfe_genomics = train(Histology~., data = genetic_data_AUC_histology, method = "rf", trControl = ctrl, tuneLength=5)
pred_rf_rfe_genomics <- predict(model_rf_rfe_genomics, test[,-1])
perf_rf_rfe_genomics <- confMat(pred_rf_rfe_genomics, test$Histology, verbose = FALSE)$all

model_rf_rfe_all = train(Histology~., data = all_rfe, method = "rf", trControl = ctrl, tuneLength=5)
pred_rf_rfe_all <- predict(model_rf_rfe_all, test[,-1])
perf_rf_rfe_all <- confMat(pred_rf_rfe_all, test$Histology, verbose = FALSE)$all


########## Support Vector Machines - Linear
model_svm_linear_rfe_radiomics = train(Histology~., data = radiomics_data_AUC_Histology, method = "svmLinear", trControl = ctrl, tuneLength=5)
pred_svm_linear_rfe_radiomics <- predict(model_svm_linear_rfe_radiomics, test[,-1])
perf_svm_linear_rfe_radiomics <- confMat(pred_svm_linear_rfe_radiomics, test$Histology, verbose = FALSE)$all

model_svm_linear_rfe_genomics = train(Histology~., data = genetic_data_AUC_histology, method = "svmLinear", trControl = ctrl, tuneLength=5)
pred_svm_linear_rfe_genomics <- predict(model_svm_linear_rfe_genomics, test[,-1])
perf_svm_linear_rfe_genomics <- confMat(pred_svm_linear_rfe_genomics, test$Histology, verbose = FALSE)$all

model_svm_linear_rfe_all = train(Histology~., data = all_rfe, method = "svmLinear", trControl = ctrl, tuneLength=5)
pred_svm_linear_rfe_all <- predict(model_svm_linear_rfe_all, test[,-1])
perf_svm_linear_rfe_all <- confMat(pred_svm_linear_rfe_all, test$Histology, verbose = FALSE)$all


########## Support Vector Machines - Radial
model_svm_radial_rfe_radiomics = train(Histology~., data = radiomics_data_AUC_Histology, method = "svmRadial", trControl = ctrl, tuneLength=5)
pred_svm_radial_rfe_radiomics <- predict(model_svm_radial_rfe_radiomics, test[,-1])
perf_svm_radial_rfe_radiomics <- confMat(pred_svm_radial_rfe_radiomics, test$Histology, verbose = FALSE)$all

model_svm_radial_rfe_genomics = train(Histology~., data = genetic_data_AUC_histology, method = "svmRadial", trControl = ctrl, tuneLength=5)
pred_svm_radial_rfe_genomics <- predict(model_svm_radial_rfe_genomics, test[,-1])
perf_svm_radial_rfe_genomics <- confMat(pred_svm_radial_rfe_genomics, test$Histology, verbose = FALSE)$all

model_svm_radial_rfe_all = train(Histology~., data = all_rfe, method = "svmRadial", trControl = ctrl, tuneLength=5)
pred_svm_radial_rfe_all <- predict(model_svm_radial_rfe_all, test[,-1])
perf_svm_radial_rfe_all <- confMat(pred_svm_radial_rfe_all, test$Histology, verbose = FALSE)$all



########## Support Vector Machines - Polynomial
model_svm_poly_rfe_radiomics = train(Histology~., data = radiomics_data_AUC_Histology, method = "svmPoly", trControl = ctrl, tuneLength=5)
pred_svm_poly_rfe_radiomics <- predict(model_svm_poly_rfe_radiomics, test[,-1])
perf_svm_poly_rfe_radiomics <- confMat(pred_svm_poly_rfe_radiomics, test$Histology, verbose = FALSE)$all

model_svm_poly_rfe_genomics = train(Histology~., data = genetic_data_AUC_histology, method = "svmPoly", trControl = ctrl, tuneLength=5)
pred_svm_poly_rfe_genomics <- predict(model_svm_poly_rfe_genomics, test[,-1])
perf_svm_poly_rfe_genomics <- confMat(pred_svm_poly_rfe_genomics, test$Histology, verbose = FALSE)$all

model_svm_poly_rfe_all = train(Histology~., data = all_rfe, method = "svmPoly", trControl = ctrl, tuneLength=5)
pred_svm_poly_rfe_all <- predict(model_svm_poly_rfe_all, test[,-1])
perf_svm_poly_rfe_all <- confMat(pred_svm_poly_rfe_all, test$Histology, verbose = FALSE)$all



########## Artificial Neural Networks
model_ann_rfe_radiomics = train(Histology~., data = radiomics_data_AUC_Histology, method = "nnet", trControl = ctrl, trace = FALSE, tuneLength=5)
pred_ann_rfe_radiomics <- predict(model_ann_rfe_radiomics, test[,-1])
perf_ann_rfe_radiomics <- confMat(pred_ann_rfe_radiomics, test$Histology, verbose = FALSE)$all

model_ann_rfe_genomics = train(Histology~., data = genetic_data_AUC_histology, method = "nnet", trControl = ctrl, trace = FALSE, tuneLength=5)
pred_ann_rfe_genomics <- predict(model_ann_rfe_genomics, test[,-1])
perf_ann_rfe_genomics <- confMat(pred_ann_rfe_genomics, test$Histology, verbose = FALSE)$all

model_ann_rfe_all = train(Histology~., data = all_rfe, method = "nnet", trControl = ctrl, trace = FALSE, tuneLength=5)
pred_ann_rfe_all <- predict(model_ann_rfe_all, test[,-1])
perf_ann_rfe_all <- confMat(pred_ann_rfe_all, test$Histology, verbose = FALSE)$all



########## XGBoost
model_xgboost_rfe_radiomics = train(Histology~., data = radiomics_data_AUC_Histology, method = "xgbTree", trControl = ctrl, tuneLength=5)
pred_xgboost_rfe_radiomics <- predict(model_xgboost_rfe_radiomics, test[,-1])
perf_xgboost_rfe_radiomics <- confMat(pred_xgboost_rfe_radiomics, test$Histology, verbose = FALSE)$all

model_xgboost_rfe_genomics = train(Histology~., data = genetic_data_AUC_histology, method = "xgbTree", trControl = ctrl, tuneLength=5)
pred_xgboost_rfe_genomics <- predict(model_xgboost_rfe_genomics, test[,-1])
perf_xgboost_rfe_genomics <- confMat(pred_xgboost_rfe_genomics, test$Histology, verbose = FALSE)$all

model_xgboost_rfe_all = train(Histology~., data = all_rfe, method = "xgbTree", trControl = ctrl, tuneLength=5)
pred_xgboost_rfe_all <- predict(model_xgboost_rfe_all, test[,-1])
perf_xgboost_rfe_all <- confMat(pred_xgboost_rfe_all, test$Histology, verbose = FALSE)$all



results <- cbind(perf_en_rfe_radiomics, perf_en_rfe_genomics, perf_en_rfe_all, 
                 perf_rf_rfe_radiomics, perf_rf_rfe_genomics, perf_rf_rfe_all, 
                 perf_svm_linear_rfe_radiomics, perf_svm_linear_rfe_genomics, perf_svm_linear_rfe_all, 
                 perf_svm_radial_rfe_radiomics, perf_svm_radial_rfe_genomics, perf_svm_radial_rfe_all, 
                 perf_svm_poly_rfe_radiomics, perf_svm_poly_rfe_genomics, perf_svm_poly_rfe_all, 
                 perf_ann_rfe_radiomics, perf_ann_rfe_genomics, perf_ann_rfe_all, 
                 perf_xgboost_rfe_radiomics, perf_xgboost_rfe_genomics, perf_xgboost_rfe_all)


colnames(results) <- c("Elastic  net radiomics", "Elastic  net genomics", "Elastic net all", 
                       "Random forest radiomics", "Random forest genomics", "Random forest all", 
                       "SVM-linear radiomics", "SVM-linear genomics", "SVM-linear all", 
                       "SVM-radial radiomics", "SVM-radial genomics", "SVM-radial all",  
                       "SVM-poly radiomics", "SVM-poly genomics",  "SVM-poly all", 
                       "ANN radiomics", "ANN genomics", "ANN all", 
                       "Xgboost radiomics", "Xgboost genomics", "Xgboost all")

write.csv(results, "Result_AUC_set.seed_123.csv") 

ERROR: Error: package or namespace load failed for 'genefilter' in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 there is no package called 'blob'


In [67]:
dim(radiomics_data_AUC_Histology[,-1])
dim(genetic_data_AUC_histology[,-1])

[1] 40  2

[1]  40 208

In [77]:
################ Boruta
library("Boruta")
radiomics <- train[,1:33]
genomics <- train[,c(1,34:5301)]

######### 1. Boruta_getImpRfZ 
boruta.radiomics_result_boruta_getImpRfZ <- Boruta(Histology ~ ., data = radiomics, getImp = getImpRfZ)
result.radiomics_result_boruta_getImpRfZ  <- boruta.radiomics_result_boruta_getImpRfZ $finalDecision
final.radiomics_result_boruta_getImpRfZ  <- result.radiomics_result_boruta_getImpRfZ [result.radiomics_result_boruta_getImpRfZ  %in% c("Confirmed")]
names(final.radiomics_result_boruta_getImpRfZ)

boruta.genomics_result_boruta_getImpRfZ <- Boruta(Histology ~ ., data = genomics, getImp = getImpRfZ)
result.genomics_result_boruta_getImpRfZ  <- boruta.genomics_result_boruta_getImpRfZ $finalDecision
final.genomics_result_boruta_getImpRfZ  <- result.genomics_result_boruta_getImpRfZ [result.genomics_result_boruta_getImpRfZ  %in% c("Confirmed")]
names(final.genomics_result_boruta_getImpRfZ)

######### 2. Boruta_getImpFerns
boruta.radiomics_result_boruta_getImpFerns<- Boruta(Histology ~ ., data = radiomics, getImp = getImpFerns)
result.radiomics_result_boruta_getImpFerns <- boruta.radiomics_result_boruta_getImpFerns$finalDecision
final.radiomics_result_boruta_getImpFerns <- result.radiomics_result_boruta_getImpFerns[result.radiomics_result_boruta_getImpFerns %in% c("Confirmed")]
names(final.radiomics_result_boruta_getImpFerns)

boruta.genomics_result_boruta_getImpFerns<- Boruta(Histology ~ ., data = genomics, getImp = getImpFerns)
result.genomics_result_boruta_getImpFerns <- boruta.genomics_result_boruta_getImpFerns$finalDecision
final.genomics_result_boruta_getImpFerns <- result.genomics_result_boruta_getImpFerns[result.genomics_result_boruta_getImpFerns %in% c("Confirmed")]
names(final.genomics_result_boruta_getImpFerns)

######### 3. Boruta_getImpXgboost
boruta.radiomics_result_boruta_getImpXgboost<- Boruta(Histology ~ ., data = radiomics, getImp = getImpXgboost)
result.radiomics_result_boruta_getImpXgboost <- boruta.radiomics_result_boruta_getImpXgboost$finalDecision
final.radiomics_result_boruta_getImpXgboost <- result.radiomics_result_boruta_getImpXgboost[result.radiomics_result_boruta_getImpXgboost %in% c("Confirmed")]
names(final.radiomics_result_boruta_getImpXgboost)

boruta.genomics_result_boruta_getImpXgboost<- Boruta(Histology ~ ., data = genomics, getImp = getImpXgboost)
result.genomics_result_boruta_getImpXgboost <- boruta.genomics_result_boruta_getImpXgboost$finalDecision
final.genomics_result_boruta_getImpXgboost <- result.genomics_result_boruta_getImpXgboost[result.genomics_result_boruta_getImpXgboost %in% c("Confirmed")]
names(final.genomics_result_boruta_getImpXgboost)

######### 4. Boruta_getImpExtraZ
boruta.radiomics_result_boruta_getImpExtraZ<- Boruta(Histology ~ ., data = radiomics, getImp = getImpExtraZ)
result.radiomics_result_boruta_getImpExtraZ <- boruta.radiomics_result_boruta_getImpExtraZ$finalDecision
final.radiomics_result_boruta_getImpExtraZ <- result.radiomics_result_boruta_getImpExtraZ[result.radiomics_result_boruta_getImpExtraZ %in% c("Confirmed")]
names(final.radiomics_result_boruta_getImpExtraZ)

boruta.genomics_result_boruta_getImpExtraZ<- Boruta(Histology ~ ., data = genomics, getImp = getImpExtraZ)
result.genomics_result_boruta_getImpExtraZ <- boruta.genomics_result_boruta_getImpExtraZ$finalDecision
final.genomics_result_boruta_getImpExtraZ <- result.genomics_result_boruta_getImpExtraZ[result.genomics_result_boruta_getImpExtraZ %in% c("Confirmed")]
names(final.genomics_result_boruta_getImpExtraZ)

######### 5. Boruta_getImpLegacyRfZ
boruta.radiomics_getImpLegacyRfZ<- Boruta(Histology ~ ., data = radiomics, getImp = getImpLegacyRfZ)
result.radiomics_getImpLegacyRfZ <- boruta.radiomics_getImpLegacyRfZ$finalDecision
final.radiomics_getImpLegacyRfZ <- result.radiomics_getImpLegacyRfZ[result.radiomics_getImpLegacyRfZ %in% c("Confirmed")]
names(final.radiomics_getImpLegacyRfZ)

boruta.genomics_getImpLegacyRfZ<- Boruta(Histology ~ ., data = genomics, getImp = getImpLegacyRfZ)
result.genomics_getImpLegacyRfZ <- boruta.genomics_getImpLegacyRfZ$finalDecision
final.genomics_getImpLegacyRfZ <- result.genomics_getImpLegacyRfZ[result.genomics_getImpLegacyRfZ %in% c("Confirmed")]
names(final.genomics_getImpLegacyRfZ)

######### 6. Boruta_getImpExtraGini
boruta.radiomics_getImpExtraGini<- Boruta(Histology ~ ., data = radiomics, getImp = getImpExtraGini)
result.radiomics_getImpExtraGini <- boruta.radiomics_getImpExtraGini$finalDecision
final.radiomics_getImpExtraGini <- result.radiomics_getImpExtraGini[result.radiomics_getImpExtraGini %in% c("Confirmed")]
names(final.radiomics_getImpExtraGini)

boruta.genomics_getImpExtraGini<- Boruta(Histology ~ ., data = genomics, getImp = getImpExtraGini)
result.genomics_getImpExtraGini <- boruta.genomics_getImpExtraGini$finalDecision
final.genomics_getImpExtraGini <- result.genomics_getImpExtraGini[result.genomics_getImpExtraGini %in% c("Confirmed")]
names(final.genomics_getImpExtraGini)

######### 7. Boruta_getImpExtraRaw
boruta.radiomics_result_boruta_getImpExtraRaw<- Boruta(Histology ~ ., data = radiomics, getImp = getImpExtraRaw)
result.radiomics_result_boruta_getImpExtraRaw <- boruta.radiomics_result_boruta_getImpExtraRaw$finalDecision
final.radiomics_result_boruta_getImpExtraRaw <- result.radiomics_result_boruta_getImpExtraRaw[result.radiomics_result_boruta_getImpExtraRaw %in% c("Confirmed")]
names(final.radiomics_result_boruta_getImpExtraRaw)

boruta.genomics_result_boruta_getImpExtraRaw<- Boruta(Histology ~ ., data = genomics, getImp = getImpExtraRaw)
result.genomics_result_boruta_getImpExtraRaw <- boruta.genomics_result_boruta_getImpExtraRaw$finalDecision
final.genomics_result_boruta_getImpExtraRaw <- result.genomics_result_boruta_getImpExtraRaw[result.genomics_result_boruta_getImpExtraRaw %in% c("Confirmed")]
names(final.genomics_result_boruta_getImpExtraRaw)




character(0)

[1] "AAK1"   "CELF1"  "GTF3A"  "SETD5"  "ZCCHC2"

character(0)

character(0)

character(0)

...

[1] "GTF3A"  "KLHL20" "SKAP2"  "ZCCHC2"

character(0)

[1] "GTF3A"  "SKAP2"  "ZCCHC2"

character(0)

[1] "GTF3A"   "LRRC37B" "RPL26"   "ZNF598"

In [76]:
install.packages("rFerns")


  There is a binary version available but the source version is later:
       binary source needs_compilation
rFerns  4.0.0  5.0.0              TRUE

  Binaries will be installed
package 'rFerns' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Asus\AppData\Local\Temp\Rtmp0eTO3Y\downloaded_packages


In [41]:
################################################### Univariate analysis ###################################################
###### Student's t-test

ttest_results_g <- colttests(as.matrix(train[,34:5301]), as.factor(train$Histology))$p.value
names_genomics <- names(train[,34:5301])
genomics_new <- cbind(names_genomics, ttest_results_g)
genomics_new <- as.data.frame(genomics_new)
genomics_new$ttest_results_g <- as.numeric(genomics_new$ttest_results_g)
genomics_new_new <- genomics_new[order(genomics_new$ttest_results_g, decreasing = FALSE),] 
genomics_new_new_new <- genomics_new_new[seq(1, 5, 1),]

genomics_dataset <- train[,34:5301]
genomics_data_t_test <- subset(genomics_dataset, select=genomics_new_new_new$names_genomics)
genetic_data_t_test_histology <- cbind(train$Histology, genomics_data_t_test)
names(genetic_data_t_test_histology)[1] <- "Histology"

########### Corr. Filter
#mtx_g = cor(genetic_data_AUC)
#drop_g = findCorrelation(mtx_g, cutoff = .9)
#drop_g = names(genetic_data_AUC)[drop_g]

#genomics_data_AUC_2 <- subset(genetic_data_AUC, select = ! names(genetic_data_AUC) %in% drop_g)
#genetic_data_AUC_histology <- cbind(train$Histology, genomics_data_AUC_2)
#names(genetic_data_AUC_histology)[1] <- "Histology"


radiomics_rfe<-radiomics_data_t_test_Histology
genomics_rfe<-genetic_data_t_test_histology
all_rfe <- cbind(radiomics_rfe, genomics_rfe[,-1])
########## Elastic Net
model_en_rfe_radiomics = train(Histology~., data = radiomics_rfe, method = "glmnet", trControl = ctrl, tuneLength = 5)
pred_en_rfe_radiomics <- predict(model_en_rfe_radiomics, test[,-1])
perf_en_rfe_radiomics <- confMat(pred_en_rfe_radiomics, test$Histology, verbose = FALSE)$all

model_en_rfe_genomics = train(Histology~., data = genomics_rfe, method = "glmnet", trControl = ctrl, tuneLength = 5)
pred_en_rfe_genomics <- predict(model_en_rfe_genomics, test[,-1])
perf_en_rfe_genomics <- confMat(pred_en_rfe_genomics, test$Histology, verbose = FALSE)$all

model_en_rfe_all = train(Histology~., data = all_rfe, method = "glmnet", trControl = ctrl, tuneLength = 5)
pred_en_rfe_all <- predict(model_en_rfe_all, test[,-1])
perf_en_rfe_all <- confMat(pred_en_rfe_all, test$Histology, verbose = FALSE)$all


########## Random Forest
model_rf_rfe_radiomics = train(Histology~., data = radiomics_rfe, method = "rf", trControl = ctrl, tuneLength = 5)
pred_rf_rfe_radiomics <- predict(model_rf_rfe_radiomics, test[,-1])
perf_rf_rfe_radiomics <- confMat(pred_rf_rfe_radiomics, test$Histology, verbose = FALSE)$all

model_rf_rfe_genomics = train(Histology~., data = genomics_rfe, method = "rf", trControl = ctrl, tuneLength = 5)
pred_rf_rfe_genomics <- predict(model_rf_rfe_genomics, test[,-1])
perf_rf_rfe_genomics <- confMat(pred_rf_rfe_genomics, test$Histology, verbose = FALSE)$all

model_rf_rfe_all = train(Histology~., data = all_rfe, method = "rf", trControl = ctrl, tuneLength = 5)
pred_rf_rfe_all <- predict(model_rf_rfe_all, test[,-1])
perf_rf_rfe_all <- confMat(pred_rf_rfe_all, test$Histology, verbose = FALSE)$all


########## Support Vector Machines - Linear
model_svm_linear_rfe_radiomics = train(Histology~., data = radiomics_rfe, method = "svmLinear", trControl = ctrl, tuneLength = 5)
pred_svm_linear_rfe_radiomics <- predict(model_svm_linear_rfe_radiomics, test[,-1])
perf_svm_linear_rfe_radiomics <- confMat(pred_svm_linear_rfe_radiomics, test$Histology, verbose = FALSE)$all

model_svm_linear_rfe_genomics = train(Histology~., data = genomics_rfe, method = "svmLinear", trControl = ctrl, tuneLength = 5)
pred_svm_linear_rfe_genomics <- predict(model_svm_linear_rfe_genomics, test[,-1])
perf_svm_linear_rfe_genomics <- confMat(pred_svm_linear_rfe_genomics, test$Histology, verbose = FALSE)$all

model_svm_linear_rfe_all = train(Histology~., data = all_rfe, method = "svmLinear", trControl = ctrl, tuneLength = 5)
pred_svm_linear_rfe_all <- predict(model_svm_linear_rfe_all, test[,-1])
perf_svm_linear_rfe_all <- confMat(pred_svm_linear_rfe_all, test$Histology, verbose = FALSE)$all


########## Support Vector Machines - Radial
model_svm_radial_rfe_radiomics = train(Histology~., data = radiomics_rfe, method = "svmRadial", trControl = ctrl, tuneLength = 5)
pred_svm_radial_rfe_radiomics <- predict(model_svm_radial_rfe_radiomics, test[,-1])
perf_svm_radial_rfe_radiomics <- confMat(pred_svm_radial_rfe_radiomics, test$Histology, verbose = FALSE)$all

model_svm_radial_rfe_genomics = train(Histology~., data = genomics_rfe, method = "svmRadial", trControl = ctrl, tuneLength = 5)
pred_svm_radial_rfe_genomics <- predict(model_svm_radial_rfe_genomics, test[,-1])
perf_svm_radial_rfe_genomics <- confMat(pred_svm_radial_rfe_genomics, test$Histology, verbose = FALSE)$all

model_svm_radial_rfe_all = train(Histology~., data = all_rfe, method = "svmRadial", trControl = ctrl, tuneLength = 5)
pred_svm_radial_rfe_all <- predict(model_svm_radial_rfe_all, test[,-1])
perf_svm_radial_rfe_all <- confMat(pred_svm_radial_rfe_all, test$Histology, verbose = FALSE)$all



########## Support Vector Machines - Polynomial
model_svm_poly_rfe_radiomics = train(Histology~., data = radiomics_rfe, method = "svmPoly", trControl = ctrl, tuneLength = 5)
pred_svm_poly_rfe_radiomics <- predict(model_svm_poly_rfe_radiomics, test[,-1])
perf_svm_poly_rfe_radiomics <- confMat(pred_svm_poly_rfe_radiomics, test$Histology, verbose = FALSE)$all

model_svm_poly_rfe_genomics = train(Histology~., data = genomics_rfe, method = "svmPoly", trControl = ctrl, tuneLength = 5)
pred_svm_poly_rfe_genomics <- predict(model_svm_poly_rfe_genomics, test[,-1])
perf_svm_poly_rfe_genomics <- confMat(pred_svm_poly_rfe_genomics, test$Histology, verbose = FALSE)$all

model_svm_poly_rfe_all = train(Histology~., data = all_rfe, method = "svmPoly", trControl = ctrl, tuneLength = 5)
pred_svm_poly_rfe_all <- predict(model_svm_poly_rfe_all, test[,-1])
perf_svm_poly_rfe_all <- confMat(pred_svm_poly_rfe_all, test$Histology, verbose = FALSE)$all




########## Artificial Neural Networks
model_ann_rfe_radiomics = train(Histology~., data = radiomics_rfe, method = "nnet", trControl = ctrl, trace = FALSE, tuneLength = 5)
pred_ann_rfe_radiomics <- predict(model_ann_rfe_radiomics, test[,-1])
perf_ann_rfe_radiomics <- confMat(pred_ann_rfe_radiomics, test$Histology, verbose = FALSE)$all

model_ann_rfe_genomics = train(Histology~., data = genomics_rfe, method = "nnet", trControl = ctrl, trace = FALSE, tuneLength = 5)
pred_ann_rfe_genomics <- predict(model_ann_rfe_genomics, test[,-1])
perf_ann_rfe_genomics <- confMat(pred_ann_rfe_genomics, test$Histology, verbose = FALSE)$all

model_ann_rfe_all = train(Histology~., data = all_rfe, method = "nnet", trControl = ctrl, trace = FALSE, tuneLength = 5)
pred_ann_rfe_all <- predict(model_ann_rfe_all, test[,-1])
perf_ann_rfe_all <- confMat(pred_ann_rfe_all, test$Histology, verbose = FALSE)$all



########## XGBoost
model_xgboost_rfe_radiomics = train(Histology~., data = radiomics_rfe, method = "xgbTree", trControl = ctrl, tuneLength = 5)
pred_xgboost_rfe_radiomics <- predict(model_xgboost_rfe_radiomics, test[,-1])
perf_xgboost_rfe_radiomics <- confMat(pred_xgboost_rfe_radiomics, test$Histology, verbose = FALSE)$all

model_xgboost_rfe_genomics = train(Histology~., data = genomics_rfe, method = "xgbTree", trControl = ctrl, tuneLength = 5)
pred_xgboost_rfe_genomics <- predict(model_xgboost_rfe_genomics, test[,-1])
perf_xgboost_rfe_genomics <- confMat(pred_xgboost_rfe_genomics, test$Histology, verbose = FALSE)$all

model_xgboost_rfe_all = train(Histology~., data = all_rfe, method = "xgbTree", trControl = ctrl, tuneLength = 5)
pred_xgboost_rfe_all <- predict(model_xgboost_rfe_all, test[,-1])
perf_xgboost_rfe_all <- confMat(pred_xgboost_rfe_all, test$Histology, verbose = FALSE)$all




results <- cbind(perf_en_rfe_radiomics, perf_en_rfe_genomics, perf_en_rfe_all, 
                 perf_rf_rfe_radiomics, perf_rf_rfe_genomics, perf_rf_rfe_all, 
                 perf_svm_linear_rfe_radiomics, perf_svm_linear_rfe_genomics, perf_svm_linear_rfe_all, 
                 perf_svm_radial_rfe_radiomics, perf_svm_radial_rfe_genomics, perf_svm_radial_rfe_all, 
                 perf_svm_poly_rfe_radiomics, perf_svm_poly_rfe_genomics, perf_svm_poly_rfe_all, 
                 perf_ann_rfe_radiomics, perf_ann_rfe_genomics, perf_ann_rfe_all, 
                 perf_xgboost_rfe_radiomics, perf_xgboost_rfe_genomics, perf_xgboost_rfe_all)


colnames(results) <- c("Elastic  net radiomics", "Elastic  net genomics", "Elastic net all", 
                       "Random forest radiomics", "Random forest genomics", "Random forest all", 
                       "SVM-linear radiomics", "SVM-linear genomics", "SVM-linear all", 
                       "SVM-radial radiomics", "SVM-radial genomics", "SVM-radial all",  
                       "SVM-poly radiomics", "SVM-poly genomics",  "SVM-poly all", 
                       "ANN radiomics", "ANN genomics", "ANN all", 
                       "Xgboost radiomics", "Xgboost genomics", "Xgboost all")

write.csv(results, "Result_Scenario9_set.seed_123_t_test_p_5.csv")
View(results)

end <- Sys.time()
end-start



ERROR: Error in colttests(as.matrix(train[, 34:5301]), as.factor(train$Histology)): could not find function "colttests"


In [38]:
end <- Sys.time()
end-start

Time difference of 48.66725 mins